In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from public_tests import *

plt.rcParams['figure.figsize']=(5.0,4.0)
plt.rcParams['image.interpolation']='nearest'
plt.rcParams['image.cmap']='gray'
np.random.seed(1)

> ### Notations 
- Superscript [l] denotes an object of the lth layer
   - eg: a[4] is the 4th layer activation ..W[5] and b[5] are the 5th layer parameters.

- superscript(i) denotes an object from the ith eg.
   - eg: x(i) is the ith training eg input
   
- subscript i denote the ith entry of a vector.
    - eg: ai[l] denotes the ith entry of the activation in layer l, assuming this is a fully connected layer
   

> ### steps of cnn

- convolution functions, including:
    - zero padding
    - convolve window
    - convolution forward
    - convolution bacward

- pooling functions:
    - pooling forward
    - create mask
    - Distribute mask
    - pooling backward
    

In [2]:
#define padding ..zero padding

def zero_pad(X,pad):
    X_pad=np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)),mode='constant',constant_values=(0,0))
    
    return X_pad

In [3]:
# #checking 
# np.random.seed(1)
# x=np.random.randn(4,3,3,2)
# x_pad=zero_pad(x,2)
# print('x.shape=\n',x_pad.shape)
# print('x[1,1]=\n',x[1,1])
# print('x_pad[1,1]=\n',x_pad[1,1])


# fig,axarr=plt.subplot(1, 2)
# axarr[0].set_title('x')
# axarr[0].imshow(x[0, :, :, 0])
# axarr[1].set_title('x_pad')
# axarr[1].imshow(x_pad[0, :, :, 0])
# zero_pad_test(zero_pad)

In [4]:
def conv_single_step(a_slice_prev,W,b):
    
    s=np.multiply(a_slice_prev,W)
    Z=np.sum(s)
    Z=Z+float(b)
    return Z

In [5]:
## forward convolution

def conv_forward(A_prev,W,b,hparameters):
    
    (m,n_H_prev,n_W_prev,n_C_prev)=A_prev.shape[0],A_prev.shape[1],A_prev.shape[2],A_prev.shape[3]
    
    
    (f,f,n_C_prev,n_C)= W.shape[1],W.shape[1],W.shape[2],W.shape[3]
    
    stride=hparameters['stride']
    pad=hparameters['pad']
    
    n_H=int(int(n_H_prev + 2*pad -f)/stride +1)
    n_W=int(int(n_W_prev + 2*pad -f)/stride +1) 
    
    #initialize zeros
    Z=np.zeros([m,n_H,n_W,n_C])
    
    #output metrics
    A_prev_pad=zero_pad(A_prev,pad)
    
    
    for i in range(m):
        a_prev_prev=A_prev_pad[i]
        
        #moving
        for h in range(n_H):
            vert_start=stride*h
            vert_end=vert_start +f
            
            for w in range(n_W):
                horiz_start=stride*w
                horiz_end=horiz_start +f
                
                for c in range(n_C):
                    a_slice_prev=A_prev_pad[i,vert_start:vert_end,horiz_start:horiz_end,:]
                    
                    
                    weights=W[:, :, :, c]
                    biases=b[:, :, :, c]
                    Z[i,h,w,c]=conv_single_step(a_slice_prev,weights,biases)
                    
    cache=(A_prev,W,b,hparameters)
    return Z,cache
                    
            

In [ ]:
## checking

In [7]:
## define max pooling without padding

def pool_forward(A_prev,hparameters,mode='max'):
    
    
    (m,n_H_prev,n_W_prev,n_C_prev)=A_prev.shape
    
    f=hparameters['f']
    stride=hparameters['stride']
    
    n_H=int(1+(n_H_prev -f)/stride)
    n_W=int(1+(n_W_prev -f)/stride) 
    n_C=n_C_prev
    
    A=np.zeros(m,n_H,n_W,n_C)
    
    for i in range(m):
        
        #moving
        for h in range(n_H):
            vert_start=stride*h
            vert_end=vert_start +f
            
            for w in range(n_W):
                horiz_start=stride*w
                horiz_end=horiz_start +f
                
                #channel
                for c in range(n_C):
                    a_prev_slice=A_prev[i]
                    
                    
                    if mode=='max':
                        A[i,h,w,c]=np.max(a_prev_slice[vert_start:vert_end,horiz_start:horiz_end,c])
                    elif mode == 'average':
                        A[i,h,w,c]=np.average(a_prev_slice[vert_start:vert_end,horiz_start:horiz_end,c])
                    
    cache=(A_prev,hparameters)
    return A,cache

In [8]:
## background convolution

def conv_backward(dZ,cache):
    
    (A_prev,W,b,hparameters)=cache  #last op that stored
    
    (m,n_H_prev,n_W_prev,n_C_prev)=A_prev.shape
    
    
    (f,f,n_C_prev,n_C)= W.shape
    
    stride=hparameters['stride']
    pad=hparameters['pad']
    
    (m,n_H,n_W,n_C)=dZ.shape
    
    
    dA_prev=np.zeros(A_prev.shape)
    dW=np.zeros(W.shape)
    db=np.zeros(b.shape)
    
    
    A_prev_pad=zero_pad(A_prev,pad)
    dA_prev_pad=zero+pad(dA_prev,pad)
    
    
    for i in range(m):
        a_prev_pad=A_prev_pad[i,:,:,:]
        da_prev_pad=dA_prev_pad[i,:,:,:]
        
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    
                    vert_start=stride*h
                    vert_end=vert_start +f
                    horiz_start=stride*w
                    horiz_end=horiz_start +f
                    
                    a_slice=a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]
                    
                    da_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:] += W[:,:,:,c] * dZ[i,h,w,c]
                    
        dA_prev[i,:,:,:]=da_prev_pad[pad:-pad,pad:-pad,:]
        
    assert(dA_prev.shape == (m,n_H_prev,n_W_prev,n_C_prev))
    return dA_prev,dW,db
        
                    
    